### Avoids scroll-in-the-scroll in the entire Notebook

In [ ]:
from IPython.display import Javascript
def resize_colab_cell():
    display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)

# Functions for **`Embeddings`** and **`Cosine Similarity`** sores

### Function to get Embeddings from Tokenized array

In [ ]:
!pip install transformers

<IPython.core.display.Javascript object>

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 131.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.9 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')


def get_embeddings(tokenized_array):
    tokenized_array_input = " ".join(tokenized_array)
    tokenized_array_encoding = tokenizer(tokenized_array_input, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        tokenized_array_embedding = model(**tokenized_array_encoding).last_hidden_state.mean(dim=1).numpy()

    return tokenized_array_embedding

<IPython.core.display.Javascript object>

### Function to get *`Cosine Similarity`*

In [ ]:
def get_cosine_similarity(JD_req_skills_embeddings, JD_req_edu_embeddings, CV_skills_embeddings, CV_edu_embeddings):
    skills_score = cosine_similarity(JD_req_skills_embeddings, CV_skills_embeddings)
    edu_score = cosine_similarity(JD_req_edu_embeddings, CV_edu_embeddings)
    return (skills_score + edu_score) / 2.0

<IPython.core.display.Javascript object>

### Function to get Cosine Similarity for Positions

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
tfidf_vectorizer = TfidfVectorizer()

<IPython.core.display.Javascript object>

In [ ]:
def get_cosine_sim_for_category(position, category):

    tfidf_matrix = tfidf_vectorizer.fit_transform([position, category])
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
    return cosine_sim[0][0]

<IPython.core.display.Javascript object>

# Preprocessing **`Job Description`**

### Importing dataset from Hugging Face

In [ ]:
!pip install datasets

<IPython.core.display.Javascript object>

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("jacob-hugging-face/job-descriptions")
print(dataset)

<IPython.core.display.Javascript object>

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['company_name', 'job_description', 'position_title', 'description_length', 'model_response'],
        num_rows: 853
    })
})


### Extracting *`Company Name`*

In [ ]:
def JD_extract_company(job_num):
    return dataset["train"][job_num]["company_name"]

<IPython.core.display.Javascript object>

### Extracting *`Position`*

In [ ]:
def JD_extract_position(job_num) :
    return dataset["train"][job_num]["position_title"]

<IPython.core.display.Javascript object>

### Extracting *`Required Skills`*

In [ ]:
def JD_extract_required_skills(job_num):
    model_response = eval(dataset["train"][job_num]["model_response"])
    return model_response["Required Skills"] if "Required Skills" in model_response else "N/A"

<IPython.core.display.Javascript object>

### Extracting *`Educational Requirements`*

In [ ]:
def JD_extract_educational_requirements(job_num):
    model_response = eval(dataset["train"][job_num]["model_response"])
    return model_response["Educational Requirements"] if "Educational Requirements" in model_response else "N/A"

<IPython.core.display.Javascript object>

# Preprocessing **`Resume Data`**
> needs attention (to mount google drive & `.zip` file upload)

### Extracting text from resume PDF

In [ ]:
!pip install PyPDF2

<IPython.core.display.Javascript object>

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.2 MB/s eta 0:00:00


In [ ]:
import PyPDF2


def extract_resume_text(filePath):
    reader = PyPDF2.PdfReader(open(filePath, "rb"))

    text = ""
    for i in range(len(reader.pages)): text += reader.pages[i].extract_text()
    return text

<IPython.core.display.Javascript object>

### Extracting *`Category (Job Role)`*

In [ ]:
def extract_category(text):
     return text.strip().split("\n")[0].strip() if text.strip() else None

<IPython.core.display.Javascript object>

### Extracting *`Education`*

In [ ]:
import re


def extract_education(text):
    education_keywords = ['Bsc', 'B. Pharmacy', 'B Pharmacy', 'Msc', 'M. Pharmacy', 'Ph.D', 'Bachelor', 'Master']
    education = []

    for word in education_keywords:
        pattern = r"(?i)\b{}\b".format(re.escape(word))
        match = re.search(pattern, text)
        if match: education.append(match.group())

    return education

<IPython.core.display.Javascript object>

### Extracting *`Skills`*

In [ ]:
!pip install spacy
!python -m spacy info
!python -m pip freeze | grep spacy
!python -m spacy download en_core_web_sm

<IPython.core.display.Javascript object>

2023-09-17 15:15:06.467607: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================== Info about spaCy ==============================

spaCy version    3.6.1                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-5.15.109+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        en_core_web_sm (3.6.0)        

en-core-web-sm @ https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.6.0/en_core_web_sm-3.6.0-py3-none-any.whl#sha256=83276fc78a70045627144786b52e1f2728ad5e29e5e43916ec37ea9c26a11212
spacy==3.6.1
spacy-legacy==3.0.12
spacy-loggers==1.0.4
2023-09-17 15:15:18.362779: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 52.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the pack

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")


def extract_skills(text):
    skills = []
    doc = nlp(text)

    for word in doc:
        if "NN" in word.tag_: skills.append(word.text)

    return list(set(skills))

<IPython.core.display.Javascript object>

### Uploading Resume PDFs as `Zip` and Extracting
> needs attention (to upload `.zip` file)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

<IPython.core.display.Javascript object>

Mounted at /content/drive


In [ ]:
import zipfile, os


# location/path of zip file in colab runtime storage
resume_folder = "/content/drive/MyDrive/Resume-to-Job-Matcher/dataset/resume_PDF_dataset.zip"

if zipfile.is_zipfile(resume_folder):
    with zipfile.ZipFile(resume_folder, 'r') as zip_ref:
        zip_ref.extractall()
        path = os.getcwd() + "/" + os.path.splitext(os.path.basename(resume_folder))[0]
else:
    print("# ERROR: Please upload a valid ZIP file with resume PDFs.")

<IPython.core.display.Javascript object>

# Getting Embeddings for JD & CV
> processing time may differ based on the dataset size

### For progress bar

In [ ]:
!pip install tqdm
from tqdm import tqdm

<IPython.core.display.Javascript object>

### For *`Job Description`*

In [ ]:
JD_req_skills_embeddings_list = []
JD_req_edu_embeddings_list = []
JD_position_list = []

jobs = 10
for JD_num in tqdm(range(jobs), desc="Processing Jobs"):
    JD_req_skills_embeddings_list.append(get_embeddings(JD_extract_required_skills(JD_num)))
    JD_req_edu_embeddings_list.append(get_embeddings(JD_extract_educational_requirements(JD_num)))
    JD_position_list.append(JD_extract_position(JD_num))

<IPython.core.display.Javascript object>

Processing Jobs: 100%|██████████| 10/10 [00:05<00:00,  1.95it/s]


### For *`Resume/CVs`*

In [ ]:
import glob, os


CV_skills_embeddings_list = []
CV_education_embeddings_list = []
CV_category_list = []

pdf_files = glob.glob(os.path.join(path, "*.pdf"))

# for cv in pdf_files:
for cv in tqdm(pdf_files, desc="Processing resume/CVs"):
    thisPDF_text = extract_resume_text(cv)

    CV_skills_embeddings_list.append(get_embeddings(extract_skills(thisPDF_text)))
    CV_education_embeddings_list.append(get_embeddings(extract_education(thisPDF_text)))
    CV_category_list.append(extract_category(thisPDF_text))

<IPython.core.display.Javascript object>

Processing resume/CVs: 100%|██████████| 2484/2484 [31:22<00:00,  1.32it/s]


# Driver Function
> processing time may differ based on the dataset size

In [ ]:
import PyPDF2, glob, os


def main():
    # outer loop (jobs)
    for JD_num in tqdm(range(jobs), desc="Progress"):

        try:
            CV_and_score = []


            # inner loop (resume/CVs)
            for CV_num, cv in enumerate(pdf_files):
                try:
                    score = (get_cosine_similarity(JD_req_skills_embeddings_list[JD_num], JD_req_edu_embeddings_list[JD_num], CV_skills_embeddings_list[CV_num], CV_education_embeddings_list[CV_num]) + get_cosine_sim_for_category(JD_position_list[JD_num], CV_category_list[CV_num])) / 2.0
                    CV_and_score.append((score[0][0], os.path.basename(cv)))
                except:
                    print(f"\n# WARNING: Problematic Resume: {os.path.basename(cv)}\n")
            # inner loop end


            CV_and_score.sort(key=lambda x: x[0], reverse=True)
            top_scores = [score for score, _ in CV_and_score[:5]]
            top_CVs = [filename for _, filename in CV_and_score[:5]]

            print(f"\nCompany: {JD_extract_company(JD_num)} ({JD_position_list[JD_num]}): \nTop 5 CVs: {top_CVs} \nCorresponding Scores: {top_scores}\n")
            # i have kept the corresponding scores as it was, you can customise it accordingly (for example, score*100 to convert into %)

        except:
            print(f"\n# WARNING: Problematic Job Description: {JD_extract_company(JD_num)} ({JD_position_list[JD_num]})\n")

        print("\n>>>>>>>\n")
        # outer loop end

<IPython.core.display.Javascript object>

In [ ]:
if __name__ == "__main__": main()

<IPython.core.display.Javascript object>

Progress:   0%|          | 0/10 [00:00<?, ?it/s]


# WARNING: Problematic Resume: 12632728.pdf



Progress:  10%|█         | 1/10 [00:05<00:53,  5.95s/it]


Company: Google (Sales Specialist): 
Top 5 CVs: ['15581242.pdf', '29184740.pdf', '15765660.pdf', '18062906.pdf', '40987524.pdf'] 
Corresponding Scores: [0.702438, 0.50382197, 0.49999958, 0.49931955, 0.49827906]


>>>>>>>


# WARNING: Problematic Resume: 12632728.pdf



...


Company: Zander Insurance Agency (Web Designer): 
Top 5 CVs: ['29524570.pdf', '21283733.pdf', '37058472.pdf', '76010167.pdf', '68240723.pdf'] 
Corresponding Scores: [0.79487634, 0.7176434, 0.60036504, 0.5923804, 0.5858183]


>>>>>>>


# WARNING: Problematic Resume: 12632728.pdf



Progress:  90%|█████████ | 9/10 [00:57<00:06,  6.44s/it]


Company: Tuff (Web Designer): 
Top 5 CVs: ['29524570.pdf', '21283733.pdf', '37058472.pdf', '76010167.pdf', '68240723.pdf'] 
Corresponding Scores: [0.7962135, 0.7188121, 0.6020087, 0.59441674, 0.58726966]


>>>>>>>


# WARNING: Problematic Resume: 12632728.pdf



Progress: 100%|██████████| 10/10 [01:02<00:00,  6.30s/it]


Company: General Dynamics Information Technology (SR. Web Designer): 
Top 5 CVs: ['29524570.pdf', '21283733.pdf', '37058472.pdf', '76010167.pdf', '39776400.pdf'] 
Corresponding Scores: [0.6076871, 0.50786495, 0.44311693, 0.43995747, 0.43256316]


>>>>>>>

